In [ ]:
# Import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats

In [ ]:
from urllib.request import urlretrieve
urlretrieve("https://raw.githubusercontent.com/blakelobato/Predicting-Asteroid-Diameter-Dash/master/model/Pred_Ast_Diam_2.csv","Pred_Ast_Diam_2.csv")

In [ ]:
from tabulate import tabulate
plt.figure()
df = pd.read_csv("Pred_Ast_Diam_2.csv")
df = df.fillna(np.nan,axis=0)
print(df.shape)
df.head(25)

In [ ]:
import pandas_profiling
df.profile_report()

Histogram Plot of target variable 

In [ ]:
#Look at the distribution of the target variable
plt.figure()
sns.histplot(df.diameter)
plt.savefig("Histogram.png")

Start of data pre-processing

In [ ]:
#Ensuring all the missing values have been taken care of previously 
df.isna().sum()

In [ ]:
# Get an idea of the different means, distributions, and values associated with the features
df.describe()

In [ ]:
# look at possibly doing a time split for this data
df.first_year_obs.describe()

In [ ]:
#Start with splitting the data into a train, validation, and test case using an 80/20 split.

from sklearn.model_selection import train_test_split
# Split into Train and Test sets
train, test = train_test_split(df, train_size=.80, test_size=0.20, random_state=42)

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, random_state=42)

train.shape, val.shape, test.shape


In [ ]:
#Get an idea of what the train dataframe now looks like (random selection of rows from the original dataframe)
train.head()

In [ ]:
#columns are the features we are using 
train.columns

In [ ]:
# Reminder to myself which columns are categorical and numeric
features = ['orbit_id', 'e', 'a', 'i', 'om', 'w', 'ma', 'n', 'tp', 'moid','moid_jup', 'class', 'producer', 'data_arc', 'n_obs_used', 'rms','diameter', 'albedo', 'diameter_sigma', 'first_year_obs','first_month_obs', 'last_obs_year', 'last_obs_month']
numeric_cols = ['e', 'a', 'i', 'om', 'w', 'ma', 'n', 'tp', 'moid','moid_jup', 'data_arc', 'n_obs_used', 'rms','diameter', 'albedo', 'diameter_sigma', 'first_year_obs','first_month_obs', 'last_obs_year', 'last_obs_month']
#categorical_cols = ['orbit_id, 'class', 'producer']
target = 'diameter'

In [ ]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [ ]:
df.corr(numeric_only=True)

Correlation HeatMap (see attached png)

In [ ]:
plt.figure(figsize=(20, 20))
heatmap = sns.heatmap(df.corr(numeric_only=True), vmin=1, vmax=-1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)
plt.savefig('myplot.png')

Scatterplots (see attached png)

In [ ]:
plt.figure()
sns.scatterplot(
    data=df, 
    x='a', 
    y='diameter')
plt.title('Scatterplot Between Asteroid Diameter and Semimajor Axis')
plt.xlabel('Semi-major Axis')
plt.ylabel('Diameter')
plt.savefig('Scatterplot Between Asteroid Diameter and Semimajor Axis.png')

In [ ]:
plt.figure()
sns.scatterplot(
    data=df, 
    x='e', 
    y='diameter')
plt.title('Scatterplot Between Asteroid Diameter and Eccentricity')
plt.xlabel('Eccentricity')
plt.ylabel('Diameter')
plt.savefig('Scatterplot Between Asteroid Diameter and Eccentricity.png')

In [ ]:
plt.figure()
sns.scatterplot(
    data=df, 
    x='n_obs_used', 
    y='diameter')
plt.title('Scatterplot Between Asteroid Diameter and Number of Observations Used')
plt.xlabel('Number of Observations Used')
plt.ylabel('Diameter')
plt.savefig('Scatterplot Between Asteroid Diameter and Number of Observations Used.png')

Baseline Model for Dataset

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# Arrange y target vectors
target = 'diameter'
y_train = train[target]
y_val = val[target]
y_test = test[target]

# Get mean baseline
print('Mean Baseline (using 0 features)')
guess = y_train.mean()

# Train Error
y_pred = [guess] * len(y_train)
mae_t = mean_absolute_error(y_train, y_pred)
mse_t = mean_squared_error(y_train, y_pred)
rmse_t = math.sqrt(mse_t)
r2_t = r2_score(y_train, y_pred)
print(f'Training MAE Error: {mae_t:.2f} km standarized')
print(f'Training MSE Error: {mse_t:.2f} km standarized')
print(f'Validation RMSE Error: {rmse_t:.2f} km standarized')
print(f'Training R^2 Error: {r2_t:.2f}%')

# Validation Error
y_pred = [guess] * len(y_val)
mae_v = mean_absolute_error(y_val, y_pred)
mse_v = mean_squared_error(y_val, y_pred)
rmse_v = math.sqrt(mse_v)
r2_val = r2_score(y_val, y_pred)
print(f'Validation MAE Error: {mae_v:.2f} km standarized ')
print(f'Validation MSE Error: {mse_v:.2f} km standarized')
print(f'Validation RMSE Error: {rmse_v:.2f} km standarized')
print(f'Validation R^2 Error: {r2_val:.2f}%')


Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline

#1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrices (already did y target vectors)
featurelr = ['n_obs_used']
X_train = train[featurelr]
X_val = val[featurelr]
X_test = test[featurelr]
print(f'Linear Regression, dependent on {featurelr}:')

# 4. Fit the model
model.fit(X_train, y_train)
y_pred_t = model.predict(X_train)
mae_t = mean_absolute_error(y_train, y_pred_t)
mse_t = mean_squared_error(y_train, y_pred_t)
rmse_t = math.sqrt(mse_t)
r2_t = r2_score(y_train, y_pred_t)
print(f'Training MAE Error: {mae_t:.2f} km standarized')
print(f'Training MSE Error: {mse_t:.2f} km standarized')
print(f'Training RMSE Error: {mse_t:.2f} km standarized')
print(f'Training R^2 Error: {r2_t:.4f}')

# 5. Apply the model to new data
y_pred_v = model.predict(X_val)
mae_v = mean_absolute_error(y_val, y_pred_v)
mse_v = mean_squared_error(y_val, y_pred_v)
rmse_v = math.sqrt(mse_v)
r2_val = r2_score(y_val, y_pred_v)
print(f'Validation MAE Error: {mae_v:.2f} km standarized ')
print(f'Validation MSE Error: {mse_v:.2f} km standarized')
print(f'Training RMSE Error: {mse_v:.2f} km standarized')
print(f'Validation R^2 Error: {r2_val:.4f}')


Decision Tree

In [ ]:
import category_encoders as ce
#from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
#from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

features = ['orbit_id', 'e', 'a', 'i', 'om', 'w', 'ma', 'n', 'tp', 'moid','moid_jup', 'class', 'producer', 'data_arc', 'n_obs_used', 'rms','diameter', 'albedo', 'diameter_sigma', 'first_year_obs','first_month_obs', 'last_obs_year', 'last_obs_month']
target = 'diameter'

# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
X_train = train.drop(columns=target)
X_val = val.drop(columns=target)

# Create a pipeline, one hot encode the low cardinality values, ordinal encode the higher cardinalaity, scale everythgin, run deceison tree regression
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True, cols=['class','producer']), 
    ce.OrdinalEncoder(cols = ['orbit_id']),
    StandardScaler(),
    DecisionTreeRegressor(criterion='friedman_mse', max_depth=15, min_samples_leaf=15, min_samples_split=4, random_state=42)
)

# fit the pipeline on training data
pipeline.fit(X_train,y_train)


y_pred_train = pipeline.predict(X_train)
y_pred_val = pipeline.predict(X_val)


print('- Training R^2 value', pipeline.score(X_train, y_train))
print('- Validation R^2 value', pipeline.score(X_val, y_val))
print(f'- Training MAE: {mean_absolute_error(y_train,y_pred_train)} km (standarized)')
print(f'- Validation MAE: {mean_absolute_error(y_val,y_pred_val)} km (standarized)')
print(f'- Training MSE: {mean_squared_error(y_train,y_pred_train)} km (standarized)')
print(f'- Validation MSE: {mean_squared_error(y_val,y_pred_val)} km (standarized)')
print(f'- Training RMSE: {math.sqrt(mean_squared_error(y_train,y_pred_train))} km (standarized)')
print(f'- Validation RMSE: {math.sqrt(mean_squared_error(y_val,y_pred_val))} km (standarized)')

Hyperoptimisation for Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

features = ['orbit_id', 'e', 'a', 'i', 'om', 'w', 'ma', 'n', 'tp', 'moid','moid_jup', 'class', 'producer', 'data_arc', 'n_obs_used', 'rms','diameter', 'albedo', 'diameter_sigma', 'first_year_obs','first_month_obs', 'last_obs_year', 'last_obs_month']
target = 'diameter'
X_train = train[features]
X_train = train.drop(columns=target)
y_train = train[target]

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True, cols=['class','producer']), 
    ce.OrdinalEncoder(cols = ['orbit_id']),
    StandardScaler(),
    DecisionTreeRegressor(random_state=42)
)

#different parameter distributions to test for the best possible combination
param_distributions = {
   'decisiontreeregressor__min_samples_leaf': [1, 3, 5, 7, 9, 10, 15], 
    'decisiontreeregressor__max_depth': [5, 7, 9, 10, 13, 15, 17, 20, 21,  25, 30], 
    'decisiontreeregressor__min_samples_split': [2, 3, 4, 5, 7],
}




# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

In [ ]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Graphing Decision Tree (hyperoptimisation was trialed and error based on hyperoptimisation)

In [ ]:
import graphviz
import os
from sklearn.tree import export_graphviz

os.environ["PATH"] += os.pathsep + r"D:\Graphviz\bin"
ord_encoder = ce.OrdinalEncoder(cols = ['orbit_id'])
X_train_ordencoded = ord_encoder.fit_transform(X_train)
X_val_ordencoded = ord_encoder.transform(X_val)

oh_encoder = ce.OneHotEncoder(use_cat_names=True, cols=['class','producer'])
X_train_encoded = oh_encoder.fit_transform(X_train_ordencoded)
X_val_encoded = oh_encoder.transform(X_val_ordencoded)

dt = pipeline.named_steps['decisiontreeregressor']

dt.fit(X_train_encoded,y_train)

encoded_columns = X_train_encoded.columns

dot_data = export_graphviz(dt, 
                           out_file=None, 
                           max_depth=7, 
                           feature_names=encoded_columns,
                           impurity=False, 
                           filled=True, 
                           proportion=True, 
                           rounded=True)   
display(graphviz.Source(dot_data))

Random Forest Regressor

In [ ]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

features = ['orbit_id', 'e', 'a', 'i', 'om', 'w', 'ma', 'n', 'tp', 'moid','moid_jup', 'class', 'producer', 'data_arc', 'n_obs_used', 'rms','diameter', 'albedo', 'diameter_sigma', 'first_year_obs','first_month_obs', 'last_obs_year', 'last_obs_month']
target = 'diameter'

# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
X_train = train.drop(columns=target)
X_val = val.drop(columns=target)

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True, cols=['class','producer']), 
    ce.OrdinalEncoder(cols = ['orbit_id']),
    StandardScaler(),
    RandomForestRegressor(n_estimators=450, max_depth=None, max_features=.77, min_samples_leaf=3, min_samples_split=3,  n_jobs=-1, random_state=42)
)

pipeline.fit(X_train,y_train)

y_pred_train = pipeline.predict(X_train)
y_pred_val = pipeline.predict(X_val)



print('Training R^2 value', pipeline.score(X_train, y_train))
print('Validation R^2 value', pipeline.score(X_val, y_val))
print(f'Training MAE: {mean_absolute_error(y_train,y_pred_train)} km (standarized)')
print(f'Validation MAE: {mean_absolute_error(y_val,y_pred_val)} km (standarized)')
print(f'Training MSE: {mean_squared_error(y_train,y_pred_train)} km (standarized)')
print(f'Validation MSE: {mean_squared_error(y_val,y_pred_val)} km (standarized)')
print(f'Training RMSE: {math.sqrt(mean_squared_error(y_train,y_pred_train))} km (standarized)')
print(f'Validation RMSE: {math.sqrt(mean_squared_error(y_val,y_pred_val))} km (standarized)')


In [ ]:
target = 'diameter'
features = train.columns.drop('diameter')

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

Cross-validation for Random Forest

In [ ]:
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

ord_encoder = ce.OrdinalEncoder(cols = ['orbit_id'])
X_train_ordencoded = ord_encoder.fit_transform(X_train)
X_val_ordencoded = ord_encoder.transform(X_val)
X_test_ordencoded = ord_encoder.transform(X_test)

oh_encoder = ce.OneHotEncoder(use_cat_names=True, cols=['class','producer'])
X_train_encoded = oh_encoder.fit_transform(X_train_ordencoded)
X_val_encoded = oh_encoder.transform(X_val_ordencoded)
X_test_encoded = oh_encoder.transform(X_test_ordencoded)

scaler = StandardScaler()
scaler.fit(X_train_encoded)
scaler.fit(X_val_encoded)
scaler.fit(X_test_encoded)

model_dt_shap = DecisionTreeRegressor(criterion='friedman_mse', max_depth=15, min_samples_leaf=15, min_samples_split=4, random_state=42)


model_dt_shap.fit(X_train_encoded,y_train)

In [ ]:
import shap
explainer = shap.TreeExplainer(model_dt_shap)
shap_values = explainer.shap_values(X_test_encoded)


Shapley Plot for Astreroid Prediction (see attached png)

In [ ]:
# summarize the effects of all the features
plt.figure()
shap.summary_plot(shap_values, X_test_encoded)
plt.savefig("Shapley.png")

Originally matplotlib was not supporting the graphs so I could not display them. I decided to save it as a png that is attached to my assessment instead. However it seems to be working now but I will leave the plots as they are, saved to the document just incase there is anything wrong with the code.